***ICI on va tester l'embedding 2. Le but est de dérouler un match et de vérifier que le vecteur d'embedding utilisé par le DQN est correct. Surtout en ce qui concerne les vecteurs de type (car j'ai l'impression qu'ils le sont pas pris en comte par le DQN dans la décision, et c'est un pb)***

In [28]:
import os
import ipynbname

chemin_notebook = ipynbname.path()
dossier_notebook = os.path.dirname(chemin_notebook)
os.chdir(dossier_notebook)
print("Répertoire actuel :", os.getcwd())

Répertoire actuel : /Users/dan2/Desktop/Télécom-master-spé/Projets_perso/Deep/Showdown_AI/My_project


In [29]:
from Utils.embedding import *

On va modifier l'embedding des types. On va réaliser un autoencodeur pour représenter la répart des types des pokes dans un espace 3 (et non 18). On garde un ordre dans ces représentations, car savoir quel type correspond à quel poké est important. Aussi on peut rajouter le nb_fainted, mais un vecteur 12 dim avec les positions des pokémons fainted, ça peut aider je pense à éviter les fausses actions.

In [30]:
#Instancier l'encodeur du type 
from Utils.autoencodeur.type.type_autoencodeur import TypeAutoencoder

encodeur_type = TypeAutoencoder(encoded_size=4)

# Charger les poids sauvegardés
encodeur_type.load_state_dict(torch.load("Utils/autoencodeur/type/type_autoencoder.pth", map_location='mps'))
encodeur_type.eval()

TypeAutoencoder(
  (encoder): Sequential(
    (0): Linear(in_features=18, out_features=12, bias=True)
    (1): ReLU()
    (2): Linear(in_features=12, out_features=4, bias=True)
  )
  (decoder): Sequential(
    (0): Linear(in_features=4, out_features=12, bias=True)
    (1): ReLU()
    (2): Linear(in_features=12, out_features=18, bias=True)
    (3): Sigmoid()
  )
)

Encodeur créé. Maintenant on remplace les types par les vecteurs encodés

In [31]:
############## DEFINIR L'EMBEDDING ##################
import numpy as np
from gymnasium.spaces import Space, Box
from poke_env.player import Gen8EnvSinglePlayer
from poke_env.data import GenData
import numpy as np
from poke_env.environment.abstract_battle import AbstractBattle
import torch
from gymnasium.spaces import Discrete, Box

# Initialiser GenData pour la génération souhaitée (par exemple, génération 8)
gen_data = GenData.from_gen(8)

# Accéder au tableau des types
type_chart = gen_data.type_chart


class embedding_Player(Gen8EnvSinglePlayer):
    def __init__(self,model_path=None, **kwargs):
        super().__init__(**kwargs)
        if model_path is not None :
            self.model = DQN.load(model_path, device="mps" if torch.backends.mps.is_available() else "cpu")
            print(f"📥 Modèle chargé depuis {model_path}")

        self.action_space = Discrete(9)  # ✅ attribut classique
    
    #Toujours mêmes valeurs de reward
    def calc_reward(self, last_battle, current_battle) -> float:
        return self.reward_computing_helper(
            current_battle, fainted_value=2.0, hp_value=1.0, victory_value=30.0
        )

    def embed_battle(self, battle )  :
        # -1 indicates that the move does not have a base power
        # or is not available
        moves_base_power = -np.ones(4)
        moves_dmg_multiplier = np.ones(4)
        moves_real_power = -np.ones(4)
        for i, move in enumerate(battle.available_moves):
            moves_base_power[i] = (
                move.base_power / 100
            )  # Simple rescaling to facilitate learning
            if move.type:
                moves_dmg_multiplier[i] = move.type.damage_multiplier(
                    battle.opponent_active_pokemon.type_1,
                    battle.opponent_active_pokemon.type_2,
                    type_chart=type_chart
                )
                moves_real_power[i] = moves_dmg_multiplier[i]*moves_base_power[i]

        pokemon_types_compressed = []
        #Pokemon types  
        pokemon_types = obtain_pokemon_types(battle)
        pokemon_types = torch.tensor(obtain_pokemon_types(battle)).view(12, 18)  # 6 pokés par team = 12 vecteurs
        with torch.no_grad():
            for i in range(12) :
                vec = pokemon_types[i].unsqueeze(0)
                encoded = encodeur_type.encoder(vec.float()) 
                pokemon_types_compressed.append(encoded.squeeze(0))

        # Final vector with 10 components
        compressed_flat = torch.cat(pokemon_types_compressed).numpy()
        final_vector = np.concatenate(
            [
                moves_real_power,
                compressed_flat,
            ]
        )

        return np.float32(final_vector)

    def describe_embedding(self) -> Space:
        low = (
            [-1] * 4 +          # real power
            [0] * 24 +         # my team types
            [0] * 24           # opponent team types
        )
        high = (
            [3] * 4 +           # real power
            [1] * 24 +         # my team types
            [1] * 24           # opponent team types
        )

        return Box(
            np.array(low, dtype=np.float32),
            np.array(high, dtype=np.float32),
            dtype=np.float32
        )
    
    def action_to_move(self, action: int, battle: AbstractBattle):
        order = super().action_to_move(action, battle)
        order.dynamax = False  # 🔥 désactive Dynamax pour toutes les actions
        return order

In [32]:
########### DEFINIR MODELE ##################
import torch
import torch.nn as nn
import torch.nn.functional as F

#Réseau perceptron à une couche cachée, sortie linéaire, f activation = Relu,
class DQNModel(nn.Module):
    def __init__(self, input_dim, n_actions):
        super(DQNModel, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=input_dim, out_channels=64, kernel_size=3, padding=1)
        self.conv12= nn.Conv1d(in_channels=64, out_channels=32, kernel_size=3, padding=1)
        self.out = nn.Linear(32, n_actions)

    def forward(self, x):
        real_power = x[:4]
        types = x[4:] 
        real_power_padded = F.pad(real_power, (0, 14))  # [18]
        real_power_padded = real_power_padded.unsqueeze(0)  # [1, 18] 
        sequence = torch.cat([real_power_padded, types], dim=0)
        sequence = sequence.T.unsqueeze(0)     
        types = types.view(12, 18) 
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        return self.out(x)

In [33]:
from stable_baselines3 import DQN
from stable_baselines3.common.env_util import make_vec_env

class EmbeddingTestPlayer(Gen8EnvSinglePlayer):

    def __init__(self, model_path="Players/embedding_3", **kwargs):
        super().__init__(**kwargs)
        self.model = DQN.load(model_path, device="mps" if torch.backends.mps.is_available() else "cpu")
        print(f"📥 Modèle chargé depuis {model_path}")
        print("Répertoire actuel :", os.getcwd())

        #Toujours mêmes valeurs de reward
    def calc_reward(self, last_battle, current_battle) -> float:
        return self.reward_computing_helper(
            current_battle, fainted_value=2.0, hp_value=1.0, victory_value=30.0
        )

    def embed_battle(self, battle )  :
        # -1 indicates that the move does not have a base power
        # or is not available
        moves_base_power = -np.ones(4)
        moves_dmg_multiplier = np.ones(4)
        moves_real_power = -np.ones(4)
        for i, move in enumerate(battle.available_moves):
            moves_base_power[i] = (
                move.base_power / 100
            )  # Simple rescaling to facilitate learning
            if move.type:
                moves_dmg_multiplier[i] = move.type.damage_multiplier(
                    battle.opponent_active_pokemon.type_1,
                    battle.opponent_active_pokemon.type_2,
                    type_chart=type_chart
                )
                moves_real_power[i] = moves_dmg_multiplier[i]*moves_base_power[i]

        pokemon_types_compressed = []
        #Pokemon types  
        pokemon_types = obtain_pokemon_types(battle)
        pokemon_types = torch.tensor(obtain_pokemon_types(battle)).view(12, 18)  # 6 pokés par team = 12 vecteurs
        with torch.no_grad():
            for i in range(12) :
                vec = pokemon_types[i].unsqueeze(0)
                encoded = encodeur_type.encoder(vec.float()) 
                pokemon_types_compressed.append(encoded.squeeze(0))

        # Final vector with 10 components
        compressed_flat = torch.cat(pokemon_types_compressed).numpy()
        final_vector = np.concatenate(
            [
                moves_real_power,
                compressed_flat,
            ]
        )
        print("vecteur embedding de ce tour : ",np.float32(final_vector),"\n")
        return np.float32(final_vector)

    def describe_embedding(self) -> Space:
        low = (
            [-1] * 4 +          # real power
            [0] * 24 +         # my team types
            [0] * 24           # opponent team types
        )
        high = (
            [3] * 4 +           # real power
            [1] * 24 +         # my team types
            [1] * 24           # opponent team types
        )

        return Box(
            np.array(low, dtype=np.float32),
            np.array(high, dtype=np.float32),
            dtype=np.float32
        )
    
    def action_to_move(self, action, battle):
        moves = battle.available_moves
        switches = battle.available_switches
        total_actions = len(moves) + len(switches)

        print(f"🔢 DQN → action={action} | #moves={len(moves)} | #switches={len(switches)} | total={total_actions}")

        if 0 <= action < len(moves):
            print("action choisie = ",self.create_order(moves[action]),"\n")
            return self.create_order(moves[action])
        elif len(moves) <= action < total_actions:
            print("action choisie = ",self.create_order(switches[action - len(moves)]),"\n")
            return self.create_order(switches[action - len(moves)])
        else:
            print("❌ Action hors bornes ! Fallback sur move aléatoire")
            return self.choose_random_move(battle)
        
    def predict(self, obs):
        obs_tensor = torch.tensor(obs, dtype=torch.float32).unsqueeze(0)
        q_values = self.model.q_net(obs_tensor)
        print(f"📊 Q-values : {q_values.detach().numpy().flatten()}")
        action = int(torch.argmax(q_values).item())
        return action

In [34]:
opponent = NoDynamaxRandomPlayer(battle_format="gen8randombattle")
eval_env = EmbeddingTestPlayer(
    battle_format="gen8randombattle", opponent=opponent, start_challenging=True
)

📥 Modèle chargé depuis Players/embedding_3
Répertoire actuel : /Users/dan2/Desktop/Télécom-master-spé/Projets_perso/Deep/Showdown_AI/My_project


In [ ]:
class Debugger : 

    """ ce serait pas mal d'avoir une classe qui permet d'observer ce qu'il y a dans obs, action mais de façon plus claire, 
    genre ça affiche les types que voit l'ordi, les puissances des mooves que voit l'ordi, pour déjà vérifier si ce qu'il observe est ce qu'on veut qu'il obsere
    /est-ce que l'observation est correcte (ex : le pokémon Félinferno est bien détecté comme un type Feu)
    puis observer quel est l'action choisie, pas avec le numéro mais la valeur (ex : attaque : flamethrower ou switch : debugant)"""
    """il faut récuperer obs, action, éventuellement info et traiter pour avoir le format qu'on veut """

In [38]:
obs = eval_env.reset()  # Lance le match (le défi est lancé automatiquement)
done = False
while not done:
    # Choisis une action (aléatoire pour le debug ou via ton agent plus tard)
    action = eval_env.action_space.sample()

    # Applique l'action
    resultat = eval_env.step(action)
    print("resultat = ",resultat,'\n')
    print("longueur résultat = ",len(resultat),'\n')

    # Affiche les infos utiles pour le debug
    print(f"Action jouée : {action}")
    print(f"Done : {done}")
    print(f"Observation (extraits) : {obs}")

    # Optionnel : afficher l'état du combat
    print(eval_env.current_battle)

    # Pause pour dérouler le match lentement (utile pour bien voir chaque étape)
    import time
    time.sleep(1)

resultat =  (array([ 0.9       ,  0.        ,  0.        ,  0.        , -2.109107  ,
        0.67357093, -1.0576273 ,  1.011922  , -1.255351  ,  1.3556677 ,
       -0.9181266 ,  0.70113677, -1.3550607 ,  1.6760488 , -0.9423771 ,
        0.6789385 , -2.0285072 ,  0.7960014 , -1.1386342 ,  0.9952727 ,
       -1.3646996 ,  1.2747602 , -0.87877595,  0.6715598 , -1.0263002 ,
        1.3125837 , -0.5289845 ,  0.42625517, -1.544438  ,  1.5230358 ,
       -1.0729465 ,  0.8510073 , -1.544438  ,  1.5230358 , -1.0729465 ,
        0.8510073 , -1.152232  ,  1.0118549 , -0.49244103,  0.46972787,
       -1.152232  ,  1.0118549 , -0.49244103,  0.46972787, -1.152232  ,
        1.0118549 , -0.49244103,  0.46972787, -1.152232  ,  1.0118549 ,
       -0.49244103,  0.46972787], dtype=float32), 0.004010152284264379, False, False, {}) 

longueur résultat =  5 

Action jouée : 16
Done : False
Observation (extraits) : (array([ 0.9       ,  0.        ,  0.        ,  0.        , -2.109107  ,
        0.67357093, -

2025-03-29 15:41:14,069 - NoDynamaxRandomP 4 - WARNING - Unmanaged move message format received - cleaned up message ['', 'move', 'p2a: Flygon', 'Outrage', 'p1a: Drifblim', '[from] lockedmove'] in battle battle-gen8randombattle-6482 turn 13


🔢 DQN → action=3 | #moves=4 | #switches=5 | total=9
action choisie =  /choose move calmmind 

vecteur embedding de ce tour :  [-1.         -1.         -1.         -1.         -1.152232    1.0118549
 -0.49244103  0.46972787 -1.255351    1.3556677  -0.9181266   0.70113677
 -1.3550607   1.6760488  -0.9423771   0.6789385  -2.0285072   0.7960014
 -1.1386342   0.9952727  -1.3646996   1.2747602  -0.87877595  0.6715598
 -1.0263002   1.3125837  -0.5289845   0.42625517 -1.544438    1.5230358
 -1.0729465   0.8510073  -1.152232    1.0118549  -0.49244103  0.46972787
 -1.920825    0.9214857  -0.793035    0.8824839  -1.3888429   1.4491565
 -0.6886382   0.7628226  -1.152232    1.0118549  -0.49244103  0.46972787
 -1.152232    1.0118549  -0.49244103  0.46972787] 

🔢 DQN → action=0 | #moves=0 | #switches=5 | total=5
action choisie =  /choose switch coalossal 

vecteur embedding de ce tour :  [ 0.          0.5         0.          0.65       -1.152232    1.0118549
 -0.49244103  0.46972787 -1.255351    1.35

RuntimeError: Battle is already finished, call reset